# Yandex

In [18]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [23]:
# Открываем файл с помощью pd.read_excel
df = pd.read_excel(r'C:\Users\79299\Desktop\DS\Report\yandex_direct.xlsx')

df_cub = pd.read_excel(r'C:\Users\79299\Desktop\DS\Report\basedata.xlsx') # Открываем файл из базы

In [24]:
# Переименовываем столбцы для единообразия
df_cub = df_cub.rename(columns={'Utm_Campaign':'Campaign',
                   'Utm_Term':'Keyword',
                    'Сумма пополнений впервые пополненных ЛК (за месяц)':'НД',
                    'Впервые пополненные ЛК':'ПЛК',
                    'Пополнения':'Все деньги',
                    'Клиенты':'ЛК'}).drop(['Utm_Source'], axis=1)
# Отбираем только те столбцы которые нам понадобятся. Можно добавлять любые
columns_list = ['Кампания',
                   'Условие показа',
                   'Показы',
                   'Клики',
                   'CTR (%)',
                   'Расход (долл.)',
                   'Ср. цена клика (долл.)',
                   'Ср. позиция показов',
                   'Ср. объём трафика',
                   'Конверсия (%)',
                   'Цена цели (долл.)',
                   'Конверсии']
# При помощи loc оставляем [все строчки(:);наш список столбцов(columns_list)]
df = df.loc[:,columns_list]
# Переименовываем столбцы для единообразия
df.rename(columns={'Кампания':'Campaign',
                   'Условие показа':'Keyword',
                   'Показы':'Impressions',
                   'Клики':'Clicks',
                   'CTR (%)':'CTR',
                   'Расход (долл.)':'Cost',
                   'Ср. цена клика (долл.)':'CPC',
                   'Ср. позиция показов':'Avg_position',
                   'Ср. объём трафика':'Avg_traffic_vol',
                   'Конверсия (%)':'Convertion_rate',
                   'Цена цели (долл.)':'CPA',
                   'Конверсии':'Convertions'}, inplace=True)

#В квадратные скобки через запятую добавляем столбцы которые нужно заменить, 
# если их больше одного. #скобки должны быть двойными
df_obj = df[['CPC', 'Avg_position', 'Convertion_rate', 'CPA', 'Convertions', 'Avg_traffic_vol']]. \
    select_dtypes(include='object')

# Исправляем тип столбцов в цикле на тип 'float64'
for i in df_obj.columns:
    df[i] = df_obj[i].replace({'-':'0'}, regex=True).astype('float64')
    
# Блок кода для приведения к единообразию написания названий кампаний в df (Выгрузка Яндекса),
# и значений utm_Campaign в df_cub

df['Campaign'] = df['Campaign'].replace({'\|\|Dec2016':''}, regex=True) #удаляем ненужные хвосты в названиях
df['Campaign'] = df['Campaign'].str.replace('(([а-яА-Я_\|0123456789]+_?)+)\|?\|', '') #оставляем правую часть названий
df['Campaign'] = df['Campaign'].str.replace('(_new)$', '') #еще раз убираем не нужный мусор

# Заполнение пропусков в столбце Keyword для кампаний ремаркетинга
a = df[df['Campaign'].str.startswith('rmkt')]['Keyword']
df.loc[a.index,['Keyword']] = '--'

# Удаление лишнего в столбцах Keyword для остальных кампаний
df_cub['Keyword'] = df_cub['Keyword'].replace({"---":''}, regex=True). \
    replace({'(%.+)':''}, regex=True)
df['Keyword'] = df['Keyword'].replace({"'---":''}, regex=True). \
    replace({'(-[a-zA-Zа-яА-Я!\+"\.0123456789]+\s?)':''}, regex=True). \
    replace({'!|\+|\[|\]|"':''}, regex=True)

# Создание нового столбца для взвешенного CTR
df['wCTR'] = np.round((df['CTR']*df['Avg_traffic_vol'])/100, 2)

# В обоих файлах создаем столбцы с уникальным ключем Кампания+ключ
df['Campaign+Key'] = pd.Series(df['Campaign'] + '+++' + df['Keyword']).str.lower().str.strip(' ')
df_cub['Campaign+Key'] = pd.Series(df_cub['Campaign'] + '+++' + df_cub['Keyword']).str.lower().str.strip(' ')

# Группируем по ключю файл из Яндекс Директа
df = df.groupby('Campaign+Key', as_index=False). \
    aggregate({'Impressions':np.sum, 
               'Clicks': np.sum,
               'wCTR': np.mean,
               'Cost': np.sum})

# Группируем по ключю файл из базы данных
df_cub = df_cub.groupby('Campaign+Key', as_index=False).aggregate({'ЛК':np.sum \
                                                                   , 'ПЛК':np.sum \
                                                                   , 'НД': np.sum \
                                                                   , 'Все деньги':np.sum \
                                                                   , 'ПЛК':np.sum})

#В обоих файлах устанавливаем наш кастомный ключ как индекс
df = df.set_index('Campaign+Key')
df_cub = df_cub.set_index('Campaign+Key')

df_f = df.join(df_cub, how='outer', on=df.index) # Объединяем таблицы
df_f = df_f.reset_index().rename(columns={'key_0':'Campaign_Key'}) # Удаляем индекс
df_f = df_f.fillna(0) # Заполняем пропуски нулями

# Разбиваем кастомный ключ обратно, на Кампанию и Ключ
df_f['Campaign'] = df_f['Campaign_Key'].apply(lambda x: x[0:x.find('+')])
df_f['Key'] = df_f['Campaign_Key'].apply(lambda x: x[x.find('+')+3:])

# Удаляем не нужное
df_f = df_f.reset_index().drop('index', axis=1)
df_f = df_f.drop('Campaign_Key', axis=1)

In [16]:
df_f.to_excel(r'C:\Users\79299\Desktop\DS\Report\report_yan_new.xlsx') # Сохраняем готовый отчет